In [21]:
import numpy as np
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import classification_report
from collections import Counter
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import make_scorer, fbeta_score

In [22]:
train_data_loaded = pd.read_csv('../data/train_data_2024-08-01.csv')

X = train_data_loaded.drop(columns=['UKATEGORIE'])
y = train_data_loaded['UKATEGORIE']

In [23]:
# StratifiedKFold initialisieren
skf = StratifiedKFold(n_splits=5)

# Variablen für Ergebnisse
fbetas_rf = []
fbetas_random = []
fbetas_mehrheit = []

In [24]:
beta = 2
fbeta_scorer= make_scorer(fbeta_score, beta=beta)

In [25]:
bal_rf_model = BalancedRandomForestClassifier(class_weight={0: 1, 1: 9}, random_state=42)

rf_scores = cross_val_score(bal_rf_model, X, y, cv=skf, scoring=fbeta_scorer)
print(f'Random Forest F-beta Score (mean): {rf_scores.mean()}')
print(f'Random Forest F-beta Score (std): {rf_scores.std()}')

# zufällige und Mehrheitsklassen-Vorhersagen
counter = 1
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Drucke die ersten 10 values von y_train
    print("y_train:", y_train.values[:10])
    # Drucke die ersten 10 values von y_test
    print("y_test:", y_test.values[:10])
    
    # Random Forest Modell trainieren
    bal_rf_model.fit(X_train, y_train)
    y_pred_rf = bal_rf_model.predict(X_test)
    print("Ersten 10 Vorhersagen mit dem Random Forest Modell:", y_pred_rf[:10])
    
    # Zufällige Vorhersagen basierend auf den Klassenwahrscheinlichkeiten erstellen
    class_counts = Counter(y_test)
    total_samples = len(y_test)
    class_probabilities = {cls: count / total_samples for cls, count in class_counts.items()}
    
    np.random.seed(42)
    y_pred_random_weighted = np.random.choice(
        list(class_probabilities.keys()),
        size=y_test.shape,
        p=list(class_probabilities.values())
    )
    
    # Drucke die ersten 10 Werte
    print("Ersten 10 Vorhersagen mit dem _random_weighted:", y_pred_random_weighted[:10])
    
    # Leistung der gewichteten zufälligen Vorhersagen bewerten
    fbeta_random_weighted = fbeta_score(y_test, y_pred_random_weighted, beta=beta, average='weighted')
    print(f"Fold Nummer {counter}: fbeta_random_weighted - Score ist {fbeta_random_weighted}")
    fbetas_random.append(fbeta_random_weighted)
    
    # Immer Mehrheitsklasse vorhersagen
    y_pred_majority = np.full(y_test.shape, y_train.mode().iloc[0])
    
    # Drucke die ersten 10 Werte
    print("Ersten 10 Vorhersagen mit dem y_pred_majority:", y_pred_majority[:10])
    
    # Leistung der Vorhersagen der Mehrheitsklasse bewerten
    fbeta_mehrheit = fbeta_score(y_test, y_pred_majority, beta=beta, average='weighted')
    print(f"Fold Nummer {counter}: fbeta_mehrheit - Score ist {fbeta_mehrheit}")
    fbetas_mehrheit.append(fbeta_mehrheit)
    
    counter += 1

# Durchschnittliche F-beta-Scores berechnen
print(f'Random Weighted Predictions F-beta Score (mean): {np.mean(fbetas_random)}')
print(f'Random Weighted Predictions F-beta Score (std): {np.std(fbetas_random)}')

print(f'Majority Class Predictions F-beta Score (mean): {np.mean(fbetas_mehrheit)}')
print(f'Majority Class Predictions F-beta Score (std): {np.std(fbetas_mehrheit)}')

    

/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/imblearn/ensemble/_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/imblearn/ensemble/_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(
/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/imblearn/ensemble/_forest.py:601: FutureWarning: The default of `bootstrap` will change from `True` to `False` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `False` to silence this warning and

Random Forest F-beta Score (mean): 0.42971101592242356
Random Forest F-beta Score (std): 0.007806205877999498
y_train: [1 1 1 1 1 1 1 1 1 1]
y_test: [0 0 0 0 0 0 0 0 0 0]


/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/imblearn/ensemble/_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/imblearn/ensemble/_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(
/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/imblearn/ensemble/_forest.py:601: FutureWarning: The default of `bootstrap` will change from `True` to `False` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `False` to silence this warning and

Ersten 10 Vorhersagen mit dem Random Forest Modell: [0 0 0 1 0 0 0 0 1 0]
Ersten 10 Vorhersagen mit dem _random_weighted: [0 1 0 0 0 0 0 1 0 0]
Fold Nummer 1: fbeta_random_weighted - Score ist 0.7463015685247019
Ersten 10 Vorhersagen mit dem y_pred_majority: [0 0 0 0 0 0 0 0 0 0]
Fold Nummer 1: fbeta_mehrheit - Score ist 0.8168620906598427
y_train: [0 0 0 0 0 0 0 0 0 0]
y_test: [1 1 1 1 1 1 1 1 1 1]


/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/imblearn/ensemble/_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/imblearn/ensemble/_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(
/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/imblearn/ensemble/_forest.py:601: FutureWarning: The default of `bootstrap` will change from `True` to `False` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `False` to silence this warning and

...

Ersten 10 Vorhersagen mit dem Random Forest Modell: [0 1 0 0 0 1 0 0 0 0]
Ersten 10 Vorhersagen mit dem _random_weighted: [0 0 0 0 0 0 1 0 0 0]
Fold Nummer 3: fbeta_random_weighted - Score ist 0.7309138770091183
Ersten 10 Vorhersagen mit dem y_pred_majority: [0 0 0 0 0 0 0 0 0 0]
Fold Nummer 3: fbeta_mehrheit - Score ist 0.8167143150750469
y_train: [0 0 0 0 0 0 0 0 0 0]
y_test: [1 1 1 1 1 1 1 1 1 1]


/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/imblearn/ensemble/_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/imblearn/ensemble/_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(
/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/imblearn/ensemble/_forest.py:601: FutureWarning: The default of `bootstrap` will change from `True` to `False` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `False` to silence this warning and

Ersten 10 Vorhersagen mit dem Random Forest Modell: [0 1 1 0 0 0 0 0 0 1]
Ersten 10 Vorhersagen mit dem _random_weighted: [0 0 0 0 0 0 1 0 0 0]
Fold Nummer 4: fbeta_random_weighted - Score ist 0.7416122431135633
Ersten 10 Vorhersagen mit dem y_pred_majority: [0 0 0 0 0 0 0 0 0 0]
Fold Nummer 4: fbeta_mehrheit - Score ist 0.8167143150750469
y_train: [0 0 0 0 0 0 0 0 0 0]
y_test: [1 1 1 1 1 1 1 1 1 1]


/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/imblearn/ensemble/_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/imblearn/ensemble/_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(
/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/imblearn/ensemble/_forest.py:601: FutureWarning: The default of `bootstrap` will change from `True` to `False` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `False` to silence this warning and

Ersten 10 Vorhersagen mit dem Random Forest Modell: [0 1 0 1 1 1 0 0 1 0]
Ersten 10 Vorhersagen mit dem _random_weighted: [0 0 0 0 0 0 1 0 0 0]
Fold Nummer 5: fbeta_random_weighted - Score ist 0.7340082584457402
Ersten 10 Vorhersagen mit dem y_pred_majority: [0 0 0 0 0 0 0 0 0 0]
Fold Nummer 5: fbeta_mehrheit - Score ist 0.8168394026197696
Random Weighted Predictions F-beta Score (mean): 0.7382213392739826
Random Weighted Predictions F-beta Score (std): 0.005439482140048987
Majority Class Predictions F-beta Score (mean): 0.8167984428179098
Majority Class Predictions F-beta Score (std): 6.918779541544925e-05


In [26]:
# Durchschnittliche Genauigkeiten berechnen
mean_scores_rf = np.mean(fbetas_rf)
mean_scores_random = np.mean(fbetas_random)
mean_scores_mehrheit = np.mean(fbetas_mehrheit)

/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [27]:
print(f"LogReg Mean score: {mean_scores_rf}")
print(f"Weighted Random Prediction Mean score: {mean_scores_random}")
print(f"Weighted Mehrheit Prediction Mean score: {mean_scores_mehrheit}")


LogReg Mean score: nan
Weighted Random Prediction Mean score: 0.7382213392739826
Weighted Mehrheit Prediction Mean score: 0.8167984428179098


In [28]:

# Klassifikationsberichte für den letzten Fold ausgeben
print("LogReg Classification Report (Last Fold):")
print(classification_report(y_test, y_pred_rf))

print("Weighted Random Prediction Classification Report (Last Fold):")
print(classification_report(y_test, y_pred_random_weighted))

LogReg Classification Report (Last Fold):
              precision    recall  f1-score   support

           0       0.89      0.67      0.77      6787
           1       0.24      0.56      0.33      1231

    accuracy                           0.65      8018
   macro avg       0.56      0.62      0.55      8018
weighted avg       0.79      0.65      0.70      8018

Weighted Random Prediction Classification Report (Last Fold):
              precision    recall  f1-score   support

           0       0.84      0.84      0.84      6787
           1       0.14      0.15      0.14      1231

    accuracy                           0.73      8018
   macro avg       0.49      0.49      0.49      8018
weighted avg       0.74      0.73      0.74      8018

